# Project Image Processing 2021
**Author : Verardo Luca, Fridez Lucas & Dos Santos Ferreira Julien**

**Course : 3252 - Image Processing**

**Profesor : M. Tièche**

This notebook contains the complete image set analyzer.

To use this notebook please refer to user documentation.

## Watershed algorithm

In [1]:
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import os, subprocess

### Constants

In [2]:
IMAGE_SET_PATH = "../dataset/test/PBC_dataset_normal_DIB/platelet"

### Function to show image

In [3]:
def show(img,title):
    plt.imshow(img)
    plt.title(title)
    plt.show()

### Watershed algorithm and counting

In [4]:
watershedCounting = 0
def watershed_image(img_path):
    global watershedCounting
    #Import and transformation of src image

    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(img,127, 255, cv2.THRESH_BINARY_INV)

    # preprocessing the image
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(binary,kernel,iterations = 1)
    hsv = cv2.cvtColor(erosion, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    _, thr = cv2.threshold(s, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel_size = 5
    blur = cv2.medianBlur(thr, kernel_size)
    area_min = 1000

    # Apply watershed
    contours, hierarchy = cv2.findContours(
        blur, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    count = []
    for x in contours:
        area = cv2.contourArea(x)
        if area > area_min:
            count.append(x)

    D = ndimage.distance_transform_edt(thr)
    localMax = peak_local_max(D, indices=False, min_distance=40, labels=thr)
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thr)
    ws = len(np.unique(labels)) - 1

    #Count number of segmentation
    # cv2.drawContours(img, count, -1, (255, 0, 0), 3)
    watershedCounting = watershedCounting + len(count)

In [5]:
for filename in os.listdir(IMAGE_SET_PATH) :
    watershed_image(f"{IMAGE_SET_PATH}/{filename}")

print("Number of blood cells segments detected = ", watershedCounting)

<ipython-input-4-5319e6b85efb>:30: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  localMax = peak_local_max(D, indices=False, min_distance=40, labels=thr)
Number of blood cells segments detected =  1169


## YOLOv5

In [9]:
# Process the same image in YOLOv5
temp_dir = "/tmp/todetect/"
yolo_project_dir = "/tmp/temp_detect"
os.system("rm -rf " + temp_dir + " " + yolo_project_dir)
# os.system("mkdir " + temp_dir )
# os.system("cp " + img_path + " /tmp/todetect/image.png")

result = subprocess.check_output(f"python3 ../detect.py --weights ../best.pt --source {IMAGE_SET_PATH} --project ../temp_detect", shell=True)
with open("../result.txt", "w") as f:
    f.write(result.decode("utf-8"))

yolo_img = cv2.imread("/tmp/temp_detect/exp/image.png")
show(yolo_img, "YOLOv5 inference")

TypeError: Image data of dtype object cannot be converted to float